In [1]:
from shared.preface import *
import shared.functions as fct
import shared.my_units as my

t = fct.s_of_z(100)
print(type(t), t)


<class 'numpy.float64'> -1.322494533419968e+18


In [2]:
z = 0
x_i = np.array([1.,2.,3.])*unit.km

t2 = fct.dPsi_dxi_NFW(x_i, z, my.rho0_NFW, my.Mvir_NFW)
print(type(t2), t2)

<class 'astropy.units.quantity.Quantity'> [-14867.28345853 -29734.56691707 -44601.8503756 ] kpc / s2


In [4]:
m_nu = 0.05 * unit.eV
m_nu_kg = m_nu.to(unit.kg, unit.mass_energy())
print(m_nu_kg)

8.91330960813949e-38 kg


In [6]:
pt = 2. * unit.kpc/
mt = 0.05 * unit.eV

fct.Fermi_Dirac()

TypeError: Fermi_Dirac() missing 1 required positional argument: 'p'